## Đặt vấn đề: Sử dụng phương pháp học máy đưa ra dự đoán kết quả dựa vào các chỉ số của đội sau trận đấu (Thắng/Hòa/Thua)

Mô hình: 

- Logistics Regression.

Trích chọn đặc trưng: 

- Dựa vào biểu đồ heatmap đầu tiên, với 2 chỉ số bất kì có độ tương quan cao, ở đây nhóm đưa ra tiêu chuẩn |correlation| >= 0.6 thì sẽ loại bỏ bớt đi 1 trong 2 chỉ số.
- Như vậy các chỉ số được loại bỏ đi sau quá trình quan sát biểu đồ được trực quan: 'xg', 'xga', 'sh', 'sot', 'pk'.

Tiền xử lý dữ liệu:

- Loại bỏ các mẫu có giá trị thuộc tính không xác định.  

Tập train, test
- Train: Các trận đấu của mùa giải 2020, 2021, 2022.
- Test: Các trận đấu của mùa giải 2023.

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [53]:
df = pd.read_csv("matches.csv")
df = df.loc[:, ['season','gf', 'ga', 'poss', 'attendance', 'dist', 'fk', 'pkatt','result']]
df = df.dropna()

# #df.loc[df['result'] != 'W', 'result'] = 'N'
#X_train, X_test, y_train, y_test = train_test_split(df[['gf', 'ga', 'poss', 'attendance', 'dist', 'fk', 'pkatt']], df['result'], test_size=0.2, random_state=42)
# Chọn các mẫu trong tập huấn luyện có giá trị năm là 2020, 2021, 2022
train = df[df['season'].isin([2020, 2021, 2022])]

# Chọn các mẫu trong tập kiểm tra có giá trị năm là 2023
test = df[df['season'] == 2023]

# Tách các mẫu huấn luyện thành đầu vào X và nhãn y
X_train = train.drop(['season','result'], axis=1)
y_train = train['result']

# Tách các mẫu kiểm tra thành đầu vào X và nhãn y
X_test = test.drop(['season','result'], axis=1)
y_test = test['result']

# Sử dụng mô hình hồi quy logistic
lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [54]:
# Đánh giá mô hình
acc = accuracy_score(y_test, y_pred)
print('Accuracy score: ', acc)

cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix: \n', cm)

cr = classification_report(y_test, y_pred)
print('Classification report: \n', cr)

Accuracy score:  0.5759637188208617
Confusion matrix: 
 [[  2  66  40]
 [  1 130  35]
 [  2  43 122]]
Classification report: 
               precision    recall  f1-score   support

           D       0.40      0.02      0.04       108
           L       0.54      0.78      0.64       166
           W       0.62      0.73      0.67       167

    accuracy                           0.58       441
   macro avg       0.52      0.51      0.45       441
weighted avg       0.54      0.58      0.50       441

